In [2]:
# import basic packages
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

import yfinance as yf

In [8]:

tickers = ['TSLA']

start_date = '2018-1-1'  # start day for lookback
end_date = '2020-1-1'  # end day for lookback

ohlc = yf.download(tickers, period="max",start=start_date, end=end_date)
ohlc.head()


data = bt.feeds.YahooFinanceCSVData(dataname='TSLA.csv') 

cerebro.adddata(data)


[*********************100%***********************]  1 of 1 completed


In [16]:
import backtrader as bt


class PrintClose(bt.Strategy):

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{txt}{dt}')
        # Print date and close

    def next(self):
        self.log('Close: ', self.dataclose[0])


# Instantiate Cerebro engine
cerebro = bt.Cerebro()

# Add data feed to Cerebro
data = bt.feeds.YahooFinanceCSVData(dataname='TSLA.csv')
cerebro.adddata(data)

# Add strategy to Cerebro
cerebro.addstrategy(PrintClose)

# Run Cerebro Engine
cerebro.run()


Close: 130.11
Close: 156.0
Close: 177.41
Close: 146.94
Close: 149.79
Close: 149.61
Close: 154.26
Close: 154.88
Close: 153.46
Close: 160.8
Close: 160.01
Close: 171.68
Close: 183.48
Close: 179.88
Close: 180.2
Close: 166.76
Close: 159.98
Close: 155.76
Close: 135.8
Close: 133.6
Close: 148.72
Close: 149.1
Close: 149.9
Close: 144.91
Close: 140.7
Close: 121.6
Close: 129.07
Close: 126.85
Close: 112.11
Close: 109.32
Close: 89.01
Close: 86.04
Close: 72.24
Close: 85.53
Close: 85.51
Close: 86.86
Close: 101.0
Close: 107.85
Close: 105.63
Close: 102.87
Close: 100.43
Close: 104.8
Close: 96.31
Close: 90.89
Close: 96.0
Close: 103.25
Close: 109.09
Close: 109.77
Close: 114.6
Close: 130.19
Close: 141.98
Close: 145.97
Close: 149.04
Close: 150.78
Close: 149.27
Close: 137.34
Close: 146.42
Close: 141.13
Close: 145.03
Close: 159.75
Close: 153.82
Close: 160.1
Close: 156.38
Close: 140.26
Close: 152.24
Close: 153.64
Close: 156.52
Close: 156.01
Close: 163.88
Close: 162.26
Close: 161.88
Close: 158.19
Close: 160.67
C

In [3]:

import argparse
import datetime

import scipy.stats

import backtrader as bt


class PearsonR(bt.ind.PeriodN):
    _mindatas = 2  # hint to the platform

    lines = ('correlation',)
    params = (('period', 20),)

    def next(self):
        c, p = scipy.stats.pearsonr(self.data0.get(size=self.p.period),
                                    self.data1.get(size=self.p.period))

        self.lines.correlation[0] = c


class MACrossOver(bt.Strategy):
    params = (
        ('ma', bt.ind.MovAv.SMA),
        ('pd1', 20),
        ('pd2', 20),
    )

    def __init__(self):
        ma1 = self.p.ma(self.data0, period=self.p.pd1, subplot=True)
        self.p.ma(self.data1, period=self.p.pd2, plotmaster=ma1)
        PearsonR(self.data0, self.data1)


def runstrat(args=None):
    args = parse_args(args)

    cerebro = bt.Cerebro()

    # Data feed kwargs
    kwargs = dict()

    # Parse from/to-date
    dtfmt, tmfmt = '%Y-%m-%d', 'T%H:%M:%S'
    for a, d in ((getattr(args, x), x) for x in ['fromdate', 'todate']):
        if a:
            strpfmt = dtfmt + tmfmt * ('T' in a)
            kwargs[d] = datetime.datetime.strptime(a, strpfmt)

    if not args.offline:
        YahooData = bt.feeds.YahooFinanceData
    else:
        YahooData = bt.feeds.YahooFinanceCSVData

    # Data feeds
    data0 = YahooData(dataname=args.data0, **kwargs)
    # cerebro.adddata(data0)
    cerebro.resampledata(data0, timeframe=bt.TimeFrame.Weeks)

    data1 = YahooData(dataname=args.data1, **kwargs)
    # cerebro.adddata(data1)
    cerebro.resampledata(data1, timeframe=bt.TimeFrame.Weeks)
    data1.plotinfo.plotmaster = data0

    # Broker
    kwargs = eval('dict(' + args.broker + ')')
    cerebro.broker = bt.brokers.BackBroker(**kwargs)

    # Sizer
    kwargs = eval('dict(' + args.sizer + ')')
    cerebro.addsizer(bt.sizers.FixedSize, **kwargs)

    # Strategy
    if True:
        kwargs = eval('dict(' + args.strat + ')')
        cerebro.addstrategy(MACrossOver, **kwargs)

    cerebro.addobserver(bt.observers.LogReturns2,
                        timeframe=bt.TimeFrame.Weeks,
                        compression=20)

    # Execute
    cerebro.run(**(eval('dict(' + args.cerebro + ')')))

    if args.plot:  # Plot if requested to
        cerebro.plot(**(eval('dict(' + args.plot + ')')))


def parse_args(pargs=None):

    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description=(
            'Gold vs SP500 from '
            'https://estrategiastrading.com/oro-bolsa-estadistica-con-python/')
    )

    parser.add_argument('--data0', required=False, default='SPY',
                        metavar='TICKER', help='Yahoo ticker to download')

    parser.add_argument('--data1', required=False, default='GLD',
                        metavar='TICKER', help='Yahoo ticker to download')

    parser.add_argument('--offline', required=False, action='store_true',
                        help='Use the offline files')

    # Defaults for dates
    parser.add_argument('--fromdate', required=False, default='2005-01-01',
                        help='Date[time] in YYYY-MM-DD[THH:MM:SS] format')

    parser.add_argument('--todate', required=False, default='2016-01-01',
                        help='Date[time] in YYYY-MM-DD[THH:MM:SS] format')

    parser.add_argument('--cerebro', required=False, default='',
                        metavar='kwargs', help='kwargs in key=value format')

    parser.add_argument('--broker', required=False, default='',
                        metavar='kwargs', help='kwargs in key=value format')

    parser.add_argument('--sizer', required=False, default='',
                        metavar='kwargs', help='kwargs in key=value format')

    parser.add_argument('--strat', required=False, default='',
                        metavar='kwargs', help='kwargs in key=value format')

    parser.add_argument('--plot', required=False, default='',
                        nargs='?', const='{}',
                        metavar='kwargs', help='kwargs in key=value format')

    return parser.parse_args(pargs)



runstrat()


usage: ipykernel_launcher.py [-h] [--data0 TICKER] [--data1 TICKER]
                             [--offline] [--fromdate FROMDATE]
                             [--todate TODATE] [--cerebro kwargs]
                             [--broker kwargs] [--sizer kwargs]
                             [--strat kwargs] [--plot [kwargs]]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"13c4e142-5d0e-4612-9fe4-903f6032a782" --shell=9017 --transport="tcp" --iopub=9019


SystemExit: 2